### Interactive IK example using `Panda` model with 3D mouse input
- Test plotting a frame interactively using a 3D mouse
- Basic numerical IK with a single target

In [1]:
import os,glfw
import numpy as np
import matplotlib.pyplot as plt
import mujoco
import mujoco_viewer
np.set_printoptions(precision=2,suppress=True,linewidth=100)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.2]


In [2]:
# setup spacemouse
import pyspacemouse
import time
pyspacemouse.close()
success = pyspacemouse.open()
import copy

SpaceMouse Wireless found
SpaceMouse Wireless found
SpaceMouse Wireless found


In [3]:
from flexIk import velIk as velIk
from flexIk import inv as inv
from flexIk import invTypes as invTypes
from flexIk.velIkSolver import SolverTypes as SolverTypes
from mujoco_utils import *

### Parse Panda model 

In [4]:
xml_path='../model/panda/franka_panda.xml'
env = MuJoCoParserClass(name='Panda',rel_xml_path=xml_path,VERBOSE=True)
model = mujoco.MjModel.from_xml_path(xml_path)
print ("[Panda] parsed.")

n_body:[18]
body_names:['world', 'panda_base', 'panda_pedestal', 'panda_link_0', 'panda_link_1', 'panda_link_2', 'panda_link_3', 'panda_link_4', 'panda_link_5', 'panda_link_6', 'panda_link_7', 'right_hand', 'panda_right_gripper', 'panda_eef', 'panda_leftfinger', 'panda_finger_joint1_tip', 'panda_rightfinger', 'panda_finger_joint2_tip']
n_joint:[9]
joint_names:['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_finger_joint1', 'panda_finger_joint2']
joint_types:[3 3 3 3 3 3 3 2 2]
joint_ranges:
[[-2.9   2.9 ]
 [-1.76  1.76]
 [-2.9   2.9 ]
 [-3.07 -0.07]
 [-2.9   2.9 ]
 [-0.02  3.75]
 [-2.9   2.9 ]
 [ 0.    0.04]
 [-0.04  0.  ]]
n_rev_joint:[7]
rev_joint_idxs:[0 1 2 3 4 5 6]
rev_joint_names:['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7']
rev_joint_min:[-2.9  -1.76 -2.9  -3.07 -2.9  -0.02 -2.9 ]
rev_joint_max:[ 2.9   1.76  2.9  -0.07  2.9   3.75  2.9 ]
rev_j

### Interactive control a 6D frame

In [5]:
env.init_viewer(viewer_title='Interactive 3D mouse test',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180, distance=3.84, elevation=-8.32, lookat=[0.02, 0.05, 0.7 ])
env.update_viewer(VIS_TRANSPARENT=True) # transparent
env.reset() # reset

# initial position for the 3D mouse
state = pyspacemouse.read()
mouse_pos_init = np.array([0.5, 0.5, 0.5])
mouse_pos = mouse_pos_init
mouse_ori_init = np.eye(3,3)
mouse_ori = copy.deepcopy(mouse_ori_init)

# scale for the axes
scale_pos = 0.001
scale_ori = 0.1

# Loop
max_tick = 10000
while (env.tick < max_tick) and env.is_viewer_alive():
    # Update the spacemouse input
    state = pyspacemouse.read()
    mouse_pos += scale_pos * np.rad2deg(np.array([-state.y, state.x, state.z]))
    mouse_ori_temp = rpy2r(scale_ori * np.array((-state.roll, -state.pitch, -state.yaw)))
    mouse_ori = np.matmul(mouse_ori_temp, mouse_ori)
    
    # Render
    env.plot_T(p=mouse_pos, R=mouse_ori,
                PLOT_AXIS=True, axis_len=0.2, axis_width=0.01,
                PLOT_SPHERE=False, sphere_r=0.05, sphere_rgba=[0,0,1,0.9])
    env.render()
    
# Close viewer
env.close_viewer()
print ("Done.")

Done.


### Test IK with single target

In [ ]:
env.init_viewer(viewer_title='IK using Panda',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180, distance=3.84, elevation=-8.32, lookat=[0.02, 0.05, 0.7 ])

# [177.75000000000003, 3.8400858992569002, -8.324999999999987, array([0.02, 0.05, 0.7 ])]
env.update_viewer(VIS_TRANSPARENT=True) # transparent
env.reset() # reset

# Set (multiple) IK targets
ik_body_name = 'panda_eef'

# set joint angles
q_init = np.array((0, 25, 0, -120, 0, 145, 0))
q = q_init*np.pi/180.0

# Open gripper (prismatic joints)
env.forward(q=q_init*np.pi/180.0, joint_idxs=env.rev_joint_idxs)
env.forward(q=[0.04,-0.04], joint_idxs=env.pri_joint_idxs)

# initial position for the 3D mouse
state = pyspacemouse.read()
mouse_pos_init = copy.deepcopy(env.data.body('panda_eef').xpos)
mouse_pos = mouse_pos_init
mouse_ori_init = copy.deepcopy(env.get_R_body(body_name='panda_eef'))
mouse_ori = mouse_ori_init
scale_pos = .0002
scale_ori = 0.01

ik_idx = 0
ik_body_names_test = ['panda_eef']

imgs,img_ticks,max_tick = [],[],1000
while (env.tick < max_tick) and env.is_viewer_alive():
    # Update the spacemouse input
    state = pyspacemouse.read()
    mouse_pos += scale_pos * np.rad2deg(np.array([-state.y, state.x, state.z]))
    mouse_ori_temp = rpy2r(scale_ori * np.array((-state.roll, -state.pitch, -state.yaw)))
    mouse_ori = np.matmul(mouse_ori_temp, mouse_ori)
    # Numerical IK
    J,err = env.get_ik_ingredients(
            body_name=ik_body_name,p_trgt=mouse_pos,R_trgt=mouse_ori,IK_P=True,IK_R=True)
    dq = env.damped_ls(J,err,stepsize=1,eps=1e-1,th=5*np.pi/180.0)
    # Update q and FK
    q = q + dq[env.rev_joint_idxs]
#     print(q)
    env.forward(q=q,joint_idxs=env.rev_joint_idxs)
    # Render
    env.plot_T(p=env.get_p_body(body_name=ik_body_name),R=env.get_R_body(body_name=ik_body_name),
                   PLOT_AXIS=True,axis_len=0.2,axis_width=0.01,
                   PLOT_SPHERE=False,sphere_r=0.05,sphere_rgba=[1,0,0,0.9])
    env.plot_T(p=mouse_pos, R=mouse_ori,
                PLOT_AXIS=True, axis_len=0.2, axis_width=0.01,
                PLOT_SPHERE=False, sphere_r=0.05, sphere_rgba=[0,0,1,0.9])
    env.plot_T(p=[0,0,0],R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0)
    env.render()
    
    
# Close viewer
env.close_viewer()
print ("Done.")

In [ ]:
print([env.viewer.cam.azimuth, env.viewer.cam.distance, env.viewer.cam.elevation, env.viewer.cam.lookat])

### Test flexIk with single target

In [ ]:
# helper function to turn joint position limits to joint velocity limits
def compute_box_constraints(q, q_min, q_max, qd_max, qdd_max, ts):
    Qd_min = np.maximum((q_min - q) / ts, -qd_max, -np.sqrt(2*qdd_max*(q - q_min)))
    Qd_max = np.minimum((q_max - q) / ts, qd_max, np.sqrt(2*qdd_max*(q_max - q)))
        
    return Qd_min, Qd_max

In [20]:
env.init_viewer(viewer_title='IK using Panda',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180, distance=3.84, elevation=-8.32, lookat=[0.02, 0.05, 0.7 ])

env.update_viewer(VIS_TRANSPARENT=True)
env.reset() # reset

# Set (multiple) IK targets
ik_body_name = 'panda_eef'

# set joint angles
q_init = np.array((0, 25, 0, -120, 0, 145, 0))
q = q_init*np.pi/180.0

# Open gripper (prismatic joints)
env.forward(q=q_init*np.pi/180.0, joint_idxs=env.rev_joint_idxs)
env.forward(q=[0.04,-0.04], joint_idxs=env.pri_joint_idxs)

# initialize the input for the flex Ik solver
numDof = 7
C = np.eye(numDof)
qMin = env.rev_joint_min
qMax = env.rev_joint_max
qdMin = np.array([-2] * numDof)
qdMax = np.array([2] * numDof)
qddMax = np.array([1] * numDof)
ts = 0.1
s = 1

# initial position for the 3D mouse
state = pyspacemouse.read()
mouse_pos_init = copy.deepcopy(env.data.body('panda_eef').xpos)
mouse_pos = mouse_pos_init
mouse_ori_init = copy.deepcopy(env.get_R_body(body_name='panda_eef'))
mouse_ori = mouse_ori_init
scale_pos_init = .0001
scale_ori_init = 0.01

ik_idx = 0
ik_body_names_test = ['panda_eef']

display_target = False

imgs,img_ticks,max_tick = [],[],10000
while (env.tick < max_tick) and env.is_viewer_alive():
    # Update the spacemouse input
    state = pyspacemouse.read()
    if (s > 0.01):
        scale_pos = scale_pos_init
        scale_ori = scale_ori_init
    else:
        scale_pos = 2 * scale_pos_init
        scale_ori = 2 * scale_ori_init
            
    mouse_pos += scale_pos * np.rad2deg(np.array([-state.y, state.x, state.z]))
    mouse_ori_temp = rpy2r(scale_ori * np.array((-state.roll, -state.pitch, -state.yaw)))
    mouse_ori = np.matmul(mouse_ori_temp, mouse_ori)

    # update box constraints
    qCurrent = q[env.rev_joint_idxs]
    dqLow = np.maximum((qMin - qCurrent) / ts, -qdMax, -np.sqrt(np.maximum(2*qddMax*(qCurrent - qMin), 0)))
    dqUpp = np.minimum((qMax - qCurrent) / ts, qdMax, np.sqrt(np.maximum(2*qddMax*(qMax - qCurrent), 0)))
    
    # Numerical IK
    J, err = env.get_ik_ingredients(
            body_name=ik_body_name,p_trgt=mouse_pos,R_trgt=mouse_ori,IK_P=True,IK_R=True)    
    
    dxGoalData = [err]
    JData = [J[:,env.rev_joint_idxs]]
        
    dq, sData, exitCode = velIk(C, dqLow, dqUpp, dxGoalData, JData, SolverTypes.ESNS_MT, invTypes.SRINV)
    s = sData[0]
            
    # Update q and FK
    q[env.rev_joint_idxs] += dq * ts
    
    # normalized joint position [0, 1]
    qNormal = np.maximum((qMax - q[env.rev_joint_idxs]), (q[env.rev_joint_idxs] - qMin)) / (qMax - qMin)
    
    if (env.tick % 50 == 0):
        if (np.any(qNormal >=  0.99)):
            print(qNormal)
        print(s)    

    env.forward(q=q,joint_idxs=env.rev_joint_idxs)
    # Render
    env.plot_T(p=env.get_p_body(body_name=ik_body_name),R=env.get_R_body(body_name=ik_body_name),
                   PLOT_AXIS=True,axis_len=0.2,axis_width=0.01,
                   PLOT_SPHERE=False,sphere_r=0.05,sphere_rgba=[1,0,0,0.9])
    if display_target:
        env.plot_T(p=mouse_pos, R=mouse_ori,
                    PLOT_AXIS=True, axis_len=0.2, axis_width=0.01,
                    PLOT_SPHERE=False, sphere_r=0.05, sphere_rgba=[0,0,1,0.9])
    env.plot_T(p=[0,0,0],R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0)
    env.render()
    
    
# Close viewer
env.close_viewer()
print ("Done.")

1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
[0.53 0.81 0.69 0.85 0.59 0.76 1.  ]
0.028743920931337236
[0.54 0.8  0.7  0.82 0.59 0.77 1.  ]
0.0
[0.54 0.8  0.7  0.82 0.59 0.77 1.  ]
0.0
[0.54 0.8  0.7  0.82 0.59 0.77 1.  ]
0.0
[0.5  0.78 0.62 0.83 0.61 0.79 1.  ]
0.0
[0.5  0.76 0.61 0.77 0.6  0.85 1.  ]
0.0
[0.5  0.76 0.61 0.77 0.6  0.85 1.  ]
0.0
[0.5  0.76 0.61 0.77 0.6  0.85 1.  ]
0.0
[0.5  0.76 0.61 0.77 0.6  0.85 1.  ]
0.0
[0.5  0.76 0.61 0.77 0.6  0.85 1.  ]
0.0
[0.8  0.7  0.72 0.57 0.76 1.   0.96]
0.0
[0.8  0.7  0.72 0.57 0.76 1.   0.96]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.79 0.75 0.73 0.55 0.78 1.   0.95]
0.0
[0.8  0.75 0.73 0.56 0.78 1.   0.95]
0.0
[0.73 0.67 0.65 0.53 0.87 1.   0.85]
0.0
[0.65 0